# Load data

In [1]:
import os
import io
data_path = "./Data"
file_name = os.listdir(data_path)
file_path = [os.path.join(data_path, name) for name in file_name]

In [2]:
print(file_path)

['./Data\\easy_sentence.txt', './Data\\gold.txt', './Data\\gold_sentence.txt', './Data\\hard_sentence.txt']


In [3]:
easy_sentence = io.open("./Data/easy_sentence.txt", encoding="utf-8").readlines()
hard_sentence = io.open("./Data/hard_sentence.txt", encoding="utf-8").readlines()

In [4]:
data_sentence = easy_sentence
data_sentence.extend(hard_sentence)

In [5]:
data_sentence[:10]

['thành phố washington có một kiến trúc rất đa dạng\n',
 'tuy nhiên vì gặp nhiều khó khăn trong cuộc sống ông dần trở nên khó tính\n',
 'khí hậu hồng kông thuộc kiểu cận nhiệt đới và chịu ảnh hưởng của gió mùa\n',
 'khoảng hơn 70 bề mặt trái đất được bao phủ bởi các đại dương nước mặn phần còn lại là các lục địa và các đảo\n',
 'đà lạt là thành phố trực thuộc tỉnh lâm đồng nằm trên cao nguyên lâm viên thuộc vùng tây nguyên việt nam\n',
 'đổi đất đai lấy hạ tầng là một trong những việc phải làm trong bối cảnh hiện nay\n',
 'cuối tuần trước tôi về quê vì gia đình có đám\n',
 'những cơn gió căng tràng vi vu khắp núi rừng đà lạt khiến những quả hồng uống no nê những giọt sương mờ\n',
 'nhưng đó là chuyện đã qua\n',
 'biến đổi khí hậu đe doạ đến con người khi nó gây bất an lương thực khan hiếm nước lũ lụt nắng nóng cực đoan thiệt hại kinh tế và di cư\n']

# Tách từ

## Longest Matching

In [6]:
import re
import unicodedata as ud

In [7]:
def syllablize(sentence):
    word = '\w+'
    non_word = '[^\w\s]'
    digits = '\d+([\.,_]\d+)+'
    
    patterns = []
    patterns.extend([word, non_word, digits])
    patterns = f"({'|'.join(patterns)})"
    
    sentence = ud.normalize('NFC', sentence)
    tokens = re.findall(patterns, sentence, re.UNICODE)
    return [token[0] for token in tokens]

In [8]:
def load_n_grams(path):
    with open(path, encoding='utf8') as f:
        words = f.read().splitlines() 
    return words

In [9]:
def longest_matching(sentence, bi_grams, tri_grams):
    syllables = syllablize(sentence)
    syl_len = len(syllables)
    
    curr_id = 0
    word_list = []
    done = False
    
    while (curr_id < syl_len) and (not done):
        curr_word = syllables[curr_id]
        if curr_id >= syl_len - 1:
            word_list.append(curr_word)
            done = True
        else:
            next_word = syllables[curr_id + 1]
            pair_word = ' '.join([curr_word.lower(), next_word.lower()])
            if curr_id >= (syl_len - 2):
                if pair_word in bi_grams:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
            else:
                next_next_word = syllables[curr_id + 2]
                triple_word = ' '.join([pair_word, next_next_word.lower()])
                if triple_word in tri_grams:
                    word_list.append('_'.join([curr_word, next_word, next_next_word]))
                    curr_id += 3
                elif pair_word in bi_grams:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
    return word_list

In [10]:
bi_grams = load_n_grams('./Vocab/vocab_bi_gram.txt')
tri_grams = load_n_grams('./Vocab/vocab_tri_gram.txt')

In [11]:
with open('Token/longest_matching_tokens.txt', 'w', encoding='utf-8') as f:
    longest_matching_sentences = []
    for sentence in data_sentence:
        word_list = longest_matching(sentence, bi_grams, tri_grams)
        longest_matching_sentences.append(' '.join(word_list))
        f.write(' '.join(word_list) + '\n')
longest_matching_sentences[0:3]

['thành_phố washington có một kiến_trúc rất đa_dạng',
 'tuy_nhiên vì gặp nhiều khó_khăn trong cuộc_sống ông dần trở_nên khó_tính',
 'khí_hậu hồng_kông thuộc kiểu cận nhiệt_đới và chịu ảnh_hưởng của gió mùa']

In [12]:
count_longest_matching_compounds = 0
for sentence in longest_matching_sentences:
    for word in sentence.split():
        if '_' in word: count_longest_matching_compounds += 1
print('Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching:', count_longest_matching_compounds)

Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching: 224


## VnCoreNLP

In [13]:
import py_vncorenlp
model = py_vncorenlp.VnCoreNLP(annotators=["wseg", "pos"], save_dir='./')

In [14]:
with open('Token/vncore_tokens.txt', 'w', encoding='utf-8') as f:
    vncore_sentences = []
    for sentence in data_sentence:
        words = model.word_segment(sentence)[0]
        vncore_sentences.append(words)
        f.write(words + '\n')
vncore_sentences[0:3]

['thành_phố washington có một kiến_trúc rất đa_dạng',
 'tuy_nhiên vì gặp nhiều khó_khăn trong cuộc_sống ông dần trở_nên khó_tính',
 'khí_hậu hồng_kông thuộc kiểu cận_nhiệt_đới và chịu ảnh_hưởng của gió_mùa']

In [15]:
count_vncore_compounds = 0
for sentence in vncore_sentences:
    for word in sentence.split():
        if '_' in word: count_vncore_compounds += 1
print('Số lượng từ ghép khi tách từ bằng thư viện VnCoreNLP:', count_vncore_compounds)

Số lượng từ ghép khi tách từ bằng thư viện VnCoreNLP: 231


## Gold Data

In [16]:
gold_sentence = io.open("./Data/gold_sentence.txt", encoding="utf-8").readlines()

In [17]:
manual_tokenize_sentences = []
for sentence in gold_sentence:
    if sentence != '\n': 
        manual_tokenize_sentences.append(sentence.strip())
manual_tokenize_sentences[0:3]

['thành_phố washington có một kiến_trúc rất đa_dạng',
 'tuy_nhiên vì gặp nhiều khó_khăn trong cuộc_sống ông dần trở_nên khó_tính',
 'khí_hậu hồng_kông thuộc kiểu cận_nhiệt_đới và chịu ảnh_hưởng của gió_mùa']

In [18]:
count_manual_tokenize_compounds = 0
for sentence in manual_tokenize_sentences:
    for word in sentence.split():
        if '_' in word: count_manual_tokenize_compounds += 1
print('Số lượng từ ghép khi tách từ thủ công:', count_manual_tokenize_compounds)

Số lượng từ ghép khi tách từ thủ công: 256


## Đánh giá

In [19]:
import pandas as pd

In [20]:
def count_correct_words(pred, source, n_grams=3):
    pred_words = pred.split()
    source_words = source.split()
    
    total_true, tp = 0, 0
    total_errors, fp = 0, 0
    
    idx = 0
    while idx < len(pred_words):
        if pred_words[idx] not in source_words[idx:(idx + n_grams)]: 
            if '_' in pred_words[idx]: fp += 1
            del pred_words[idx]
            total_errors += 1
        else: idx += 1
    
    idx = 0
    while idx < len(source_words):
        if source_words[idx] not in pred_words[idx:(idx + n_grams)]: 
            del source_words[idx]
        else: idx += 1
    
    if len(pred_words) < len(source_words): words = pred_words
    else: words = source_words
    
    for idx in range (len(words)):
        if pred_words[idx] == source_words[idx]:
            if '_' in pred_words[idx]: tp += 1 
            total_true += 1
                    
    return total_true, total_errors, tp, fp

In [21]:
def tokenize_evaluation(pred, source, n_grams=3):
    total_true = 0
    total_errors = 0
    total_words = 0
    
    pred_tp = 0
    pred_fp = 0
    
    for pred_sentence, source_sentence in zip(pred, source):
        total_words += len(source_sentence.split())
        if pred_sentence != source_sentence:
            true, error, tp, fp = count_correct_words(pred_sentence, source_sentence, n_grams)
            total_true += true 
            total_errors += error
            pred_tp += tp
            pred_fp += fp
        else:
            for word in source_sentence.split():
                if '_' in word:
                    pred_tp += 1
                total_true += 1
                    
    accuracy = total_true / total_words
    precision = pred_tp / (pred_tp + pred_fp)
    recall = pred_tp / count_manual_tokenize_compounds
    f1 = 2 * precision * recall / (precision + recall)
    return {
        'Accuracy': accuracy, 
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'True Positive': pred_tp, 
        'False Positive': pred_fp,
        'Total True': total_true, 
        'Total Errors': total_errors
    }

In [22]:
longest_matching_evaluation = tokenize_evaluation(longest_matching_sentences, manual_tokenize_sentences)
vncore_evaluation = tokenize_evaluation(vncore_sentences, manual_tokenize_sentences)
pd.DataFrame(
    [longest_matching_evaluation, vncore_evaluation], 
    index = ['Longest Matching', 'VnCoreNLP']
).astype(object).T

,Longest Matching,VnCoreNLP
Accuracy,0.904762,0.914286
Precision,0.950893,0.943723
Recall,0.832031,0.851562
F1,0.8875,0.895277
True Positive,213,218
False Positive,11,13
Total True,665,672
Total Errors,99,81


# Train model

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
gold_corpus = io.open("./Data/gold.txt", encoding="utf-8").readlines()
gold_corpus[:10]

['thành_phố/N washington/N có/V 1/M kiến_trúc/N rất/R đa_dạng/A\n',
 'tuy_nhiên/C vì/C gặp/V nhiều/A khó_khăn/N trong/E cuộc_sống/N ông/N dần/R trở_nên/V khó_tính/A\n',
 'khí_hậu/N hồng_kông/N thuộc/V kiểu/N cận_nhiệt_đới/N và/C chịu/V ảnh_hưởng/V của/E gió_mùa/N\n',
 'khoảng/A hơn/A 70/M bề_mặt/N trái_đất/N được/V bao_phủ/V bởi/E các/D đại_dương/N nước_mặn/N phần/N còn_lại/V là/V các/D lục_địa/N và/C các/D đảo/N\n',
 'đà_lạt/N là/V thành_phố/N trực_thuộc/V tỉnh/N lâm_đồng/N nằm/V trên/E cao_nguyên/N lâm_viên/N thuộc/V vùng/N tây_nguyên/N việt_nam/N\n',
 'đổi/V đất_đai/N lấy/V hạ_tầng/N là/V 1/M trong/E những/D việc/N phải/V làm/V trong/E bối_cảnh/N hiện_nay/N\n',
 'cuối/N tuần/N trước/N tôi/P về/V quê/N vì/E gia_đình/N có/V đám/N\n',
 'những/D cơn_gió/N căng_tràng/A vi_vu/V khắp/A núi_rừng/N đà_lạt/N khiến/V những/D quả/N hồng/N uống/V no_nê/A những/D giọt_sương/N mờ/A\n',
 'nhưng/C đó/P là/V chuyện/N đã/R qua/V\n',
 'biến_đổi/N khí_hậu/N đe_doạ/V đến/E con_người/N khi/C nó/P gây/V bấ

## Vocab

In [25]:
vocab = load_n_grams('./Vocab/vocab.txt')

In [26]:
vocab_dict = {}
index = 0
for word in sorted(vocab): 
    if word not in vocab_dict: 
        vocab_dict[word] = index  
        index += 1

In [27]:
print('Số lượng từ vựng:', len(vocab_dict.values()))

Số lượng từ vựng: 23800


## Train test split

In [28]:
train, test = train_test_split(gold_corpus, train_size=0.8, random_state=23, shuffle=True)

In [29]:
train[:10]

['những/D website/N được/V thiết_kế/V đẹp_mắt/A với/C bố_cục/N hài_hoà/A và/C tỉ_lệ/N cân_đối/A sẽ/R chiếm/V được/R thiện_cảm/N của/E người_dùng/N\n',
 'việc/N tối_ưu/V tốc_độ/N tải/V trang/N luôn/R là/V nhiệm_vụ/N được/V đặt/V lên/V hàng_đầu/N\n',
 'tiếng_anh/N của/E tôi/P rất/R kém/A vì_vậy/C tôi/P cần/V cải_thiện/V nhiều/A hơn/A\n',
 'cà_phê/N ở/E đây/P làm/V tôi/P phê/V tận/V nóc/N\n',
 'con_người/N chỉ/R đơn_giản/A là/V những/D cỗ_máy/N vô_cùng/R phức_tạp/A\n',
 'công_việc/N của/E tôi/P không/R hẳn/R là/V dễ/A nhưng/C dường_như/I tôi/P đang/R suy_nghĩ/V nhiều/A về/E nó/P\n',
 'người_dùng/N có_thể/R xem/V và/C chỉ/V rõ/A thông_tin/N nào/P được/V thu_thập/V để/E thực_thi/V 1_số/D chế_định/N về/E quyền/N riêng_tư/A\n',
 'tôi/P đang/R làm/V bài_tập/N gán/V nhãn/N từ_loại/N trong/E từ_điển/N\n',
 'khoảng/A hơn/A 70/M bề_mặt/N trái_đất/N được/V bao_phủ/V bởi/E các/D đại_dương/N nước_mặn/N phần/N còn_lại/V là/V các/D lục_địa/N và/C các/D đảo/N\n',
 '1/M nghề/N cho/E chín/A còn/R hơn/A ch

In [30]:
def preprocess(vocab, data):
    num_regex = re.compile(r'\d+([.]\d+)?')
    words = []
    words_pos = []
    for line in data:
        line_split = line.split()
        words.append("<s>")
        words_pos.append(("<s>","<s>"))
        for index, word in enumerate(line_split):
            w, pos = word.split('/')
            if num_regex.fullmatch(w):
                w = "<digit>"
            words_pos.append((w, pos))
            if w not in vocab:
                w = '<unk>'
            words.append(w)
    return words, words_pos

In [31]:
train_words, train_words_pos = preprocess(vocab_dict, train)
test_words, test_words_pos = preprocess(vocab_dict, test)

In [32]:
print('Số lượng từ trong tập train_gold:', len(train_words))
print('Số lượng từ trong tập test_gold:', len(test_words))

Số lượng từ trong tập train_gold: 611
Số lượng từ trong tập test_gold: 186


In [33]:
print('Các từ không nằm trong vocabs', end=': ')
for word, word_pos in zip(test_words, test_words_pos):
    if word == '<unk>': 
        print(word_pos[0], end=', ')

Các từ không nằm trong vocabs: lâm_viên, nhìn_thấy, bàn_phím, ngón_tay, visual_designer, mọi_thứ, 

## Training

In [34]:
from collections import defaultdict

In [ ]:
def emission_prob(word, tag, train_bag)


In [35]:
def create_dictionaries(data, vocab):
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    prev_tag = ""
    for i, word_tag in enumerate(data):
        word, tag = word_tag
        if i == 0:
            prev_tag = tag
            continue
        transition_counts[(prev_tag, tag)] += 1
        emission_counts[(tag, word)] += 1
        tag_counts[tag] += 1
        prev_tag = tag
    return transition_counts, emission_counts, tag_counts

In [36]:
transition_counts, emission_counts, tag_counts = create_dictionaries(train_words_pos, vocab_dict)

In [37]:
states = sorted(tag_counts.keys())
print('Số nhãn:', len(states))
print(states)

Số nhãn: 11
['<s>', 'A', 'C', 'D', 'E', 'I', 'M', 'N', 'P', 'R', 'V']


In [38]:
print("Transition examples: ")
for example in list(transition_counts.items())[:10]:
    print(example)

Transition examples: 
(('<s>', 'D'), 4)
(('D', 'N'), 18)
(('N', 'V'), 42)
(('V', 'V'), 27)
(('V', 'A'), 17)
(('A', 'C'), 4)
(('C', 'N'), 5)
(('N', 'A'), 15)
(('A', 'R'), 3)
(('R', 'V'), 24)


In [39]:
print("Emission examples: ")
for example in list(emission_counts.items())[:10]:
    print (example)

Emission examples: 
(('D', 'những'), 10)
(('N', 'website'), 1)
(('V', 'được'), 6)
(('V', 'thiết_kế'), 1)
(('A', 'đẹp_mắt'), 1)
(('C', 'với'), 1)
(('N', 'bố_cục'), 1)
(('A', 'hài_hoà'), 1)
(('C', 'và'), 7)
(('N', 'tỉ_lệ'), 1)


In [40]:
def predict_pos(words_pos, emission_counts, vocab_dict, states):
    num_correct = 0
    all_words = set(emission_counts.keys())
    
    for word_pos in words_pos: 
        word, true_label = word_pos
        count_final = 0
        pos_final = ''
        if word not in vocab_dict: 
            continue
        
        for pos in states:
            if (pos, word) not in emission_counts: 
                continue
            count = emission_counts[(pos, word)]
            
            if count > count_final:
                count_final = count
                pos_final = pos
        if pos_final == true_label: num_correct += 1
    accuracy = num_correct / len(words_pos)
    return accuracy

In [41]:
accuracy = predict_pos(train_words_pos, emission_counts, vocab_dict, states)
print('Độ chính xác trên tập train:', accuracy)

Độ chính xác trên tập train: 0.9754500818330606


In [42]:
accuracy = predict_pos(test_words_pos, emission_counts, vocab_dict, states)
print('Độ chính xác trên tập test:', accuracy)

Độ chính xác trên tập test: 0.5161290322580645


### Hidden Markov

In [43]:
import numpy as np
import math

#### Transition matrix A

In [44]:
def create_transition_matrix(alpha, tag_counts, transition_counts):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(all_tags)
    
    A = np.zeros((num_tags, num_tags))
    trans_keys = set(transition_counts.keys())
    
    for i in range(num_tags):
        for j in range(num_tags):
            count = 0
            key = (all_tags[i], all_tags[j])
            if key in transition_counts: 
                count = transition_counts[key]
            count_prev_tag = tag_counts[all_tags[i]]
            A[i, j] = (count + alpha) / (count_prev_tag + alpha * num_tags)
    return A

In [45]:
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)

#### Emission matrix B

In [46]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocabs):
    all_tags = sorted(tag_counts.keys())
    num_tags = len(tag_counts)
    num_words = len(vocabs)
    
    B = np.zeros((num_tags, num_words))
    emis_keys = set(list(emission_counts.keys()))
    
    for i in range(num_tags):
        for j in range(num_words):
            count = 0
            key = (all_tags[i], vocabs[j])
            if key in emission_counts.keys(): 
                count = emission_counts[key]
            count_tag = tag_counts[all_tags[i]]
            B[i, j] = (count + alpha) / (count_tag + alpha * num_words)
    return B

In [47]:
B = create_emission_matrix(alpha, tag_counts, emission_counts, list(vocab_dict))

#### Viterbi

In [48]:
A = np.array([sublist[1:].tolist() for sublist in A])
B = B[1:]

In [49]:
def viterbi_initialize(states, tag_counts, A, B, corpus, vocab_dict):
    num_tags = len(states)
    s_idx = 0
    
    best_probs = np.zeros((num_tags, len(corpus)))
    best_paths = np.zeros((num_tags, len(corpus)), dtype=int)
    
    for i in range(num_tags):
        if A[s_idx, i - 1] == 0: best_probs[i, 0] = float('-inf')
        else: 
            index = vocab_dict[corpus[0]]
            # best_probs[i, 0] = math.log(A[s_idx, i]) + math.log(B[i, index])
            best_probs[i, 0] = math.log(A[s_idx, i - 1]) + math.log(B[i - 1, index])
    return best_probs, best_paths

In [50]:
best_probs_train, best_paths_train = viterbi_initialize(states, tag_counts, A, B, train_words, vocab_dict)
best_probs_test, best_paths_test = viterbi_initialize(states, tag_counts, A, B, test_words, vocab_dict)

In [51]:
def viterbi_forward(A, B, corpus, best_probs, best_paths, vocab_dict):
    num_tags = best_probs.shape[0]
    
    for i in range(1, len(corpus)):         
        for j in range(num_tags):
            best_prob_i = float('-inf')
            best_path_i = None
            
            for k in range(num_tags):
                index = vocab_dict[corpus[i]]
                # prob = best_probs[k, i - 1] + math.log(A[k, j]) + math.log(B[j, index])
                prob = best_probs[k, i - 1] + math.log(A[k, j - 1]) + math.log(B[j - 1, index])

                if prob > best_prob_i:
                    best_prob_i = prob
                    best_path_i = k
                    
            best_probs[j, i] = best_prob_i
            best_paths[j, i] = best_path_i
            
    return best_probs, best_paths

In [52]:
best_probs_train, best_paths_train = viterbi_forward(A, B, train_words, best_probs_train, best_paths_train, vocab_dict)
best_probs_test, best_paths_test = viterbi_forward(A, B, test_words, best_probs_test, best_paths_test, vocab_dict)

In [53]:
new_states = states
new_states.remove("<s>")

In [54]:
def viterbi_backward(best_probs, best_paths, states):
    m = best_paths.shape[1] 
    z = [None] * m
    pred = [None] * m
    
    best_prob_for_last_word = float('-inf')
    num_tags = best_probs.shape[0]
    
    for k in range(num_tags):
        if best_probs[k, m - 1] > best_prob_for_last_word:
            best_prob_for_last_word = best_probs[k, m - 1]
            z[m - 1] = k
            
    pred[m - 1] = states[z[m - 1]-1]
    for i in range(m - 1, -1, -1):
        z[i - 1] = best_paths[z[i], i]
        pred[i - 1] = states[z[i - 1]-1]
    return pred

In [55]:
train_pred = viterbi_backward(best_probs_train, best_paths_train, new_states)
test_pred = viterbi_backward(best_probs_test, best_paths_test, new_states)
# m = len(test_pred)

# print(f'Dự đoán cho test_pred[0:{m - 1}]:')
print(test_words[:10])
print(test_pred[:10])


['<s>', 'đà_lạt', 'là', 'thành_phố', 'trực_thuộc', 'tỉnh', 'lâm_đồng', 'nằm', 'trên', 'cao_nguyên']
['D', 'N', 'V', 'N', 'V', 'P', 'R', 'V', 'E', 'P']


## Test

In [56]:
for word, tag in zip(test_words, test_pred):
    if word == '<s>': 
        print()
    else: 
        print(f'{word}/{tag}', end=' ')


đà_lạt/N là/V thành_phố/N trực_thuộc/V tỉnh/P lâm_đồng/R nằm/V trên/E cao_nguyên/P <unk>/R thuộc/V vùng/P tây_nguyên/R việt_nam/V 
song_song/R là/V <digit>/M cửa_sổ/N <digit>/M người/N ngồi/V trong/E cửa_sổ/P song_song/R 
nó/P có/V đầy_đủ/E các/D nhân_vật/N được/V phát_triển/V chuẩn/A và/C <digit>/M bối_cảnh/N trò_chơi/P vô_cùng/R sống_động/V 
nhưng/C đó/P là/V chuyện/P đã/R qua/V 
nếu_như/R là/V trước_đây/P chắc/V tôi/P sẽ/R sợ/V không/R muốn/V ai/P <unk>/V phần/N cơ_thể/V đó/P 
sau/N trận/C mưa_lũ/I lịch_sử/P đà_nẵng/R giá/V rau/P nhích/R lên/V chính_quyền/P cảnh_báo/R không/R lợi_dụng/V nâng/P giá/R 
gõ/P <unk>/R bằng/V <digit>/M <unk>/N sẽ/R giúp/V bạn/P có/V tốc_độ/N gõ/R nhanh/A hơn/A 
mỗi/D buổi/N chiều/N tôi/P thường/R dành/V thời_gian/N để/E tập/P gym/V hoặc/P đá/R bóng/A và/C tôi/P đang/R cố_gắng/V để/E duy_trì/V nó/P 
sứ_mệnh/V của/E <unk>/P là/V mang/V đến/E những/D trải_nghiệm/N thú_vị/A và/C độc_đáo/V cho/E người_dùng/N 
điều/N quan_trọng/R là/V bạn/P phải/V có/V mục_tiê

In [57]:
from sklearn.metrics import classification_report
def report(pred, gold):
    y_pred = []
    y_true = []

    for prediction, word_tag in zip(pred, gold):
        word, tag = word_tag
        y_pred.append(prediction)
        y_true.append(tag)
        
    print(classification_report(y_pred, y_true))
    return y_pred, y_true

In [58]:
print('Kết quả của mô hình Hidden Markov kết hợp thuật toán Viterbi trên tập train:\n')
y_pred, y_true_train = report(train_pred, train_words_pos)

Kết quả của mô hình Hidden Markov kết hợp thuật toán Viterbi trên tập train:

              precision    recall  f1-score   support

         <s>       0.00      0.00      0.00         0
           A       1.00      0.96      0.98        53
           C       0.94      0.83      0.88        18
           D       1.00      0.86      0.92        21
           E       0.98      0.76      0.86        72
           I       1.00      1.00      1.00         2
           M       1.00      0.63      0.77        19
           N       0.98      0.99      0.98       185
           P       0.97      0.90      0.94        42
           R       0.98      0.93      0.95        45
           V       0.99      0.88      0.93       154

    accuracy                           0.90       611
   macro avg       0.89      0.80      0.84       611
weighted avg       0.98      0.90      0.94       611



c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
print('Kết quả của mô hình Hidden Markov kết hợp thuật toán Viterbi trên tập test:\n')
y_pred, y_true_test = report(test_pred, test_words_pos)

Kết quả của mô hình Hidden Markov kết hợp thuật toán Viterbi trên tập test:

              precision    recall  f1-score   support

         <s>       0.00      0.00      0.00         0
           A       0.41      0.78      0.54         9
           C       0.56      0.71      0.63         7
           D       1.00      0.75      0.86         4
           E       0.75      0.75      0.75        12
           I       0.00      0.00      0.00         1
           M       1.00      1.00      1.00         5
           N       0.34      1.00      0.51        18
           P       1.00      0.33      0.49        40
           R       1.00      0.36      0.53        36
           V       0.77      0.67      0.71        54
           X       0.00      0.00      0.00         0

    accuracy                           0.59       186
   macro avg       0.57      0.53      0.50       186
weighted avg       0.80      0.59      0.61       186



c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Rec

In [71]:
test_sentence = "tôi chơi đá bóng"
tokens = longest_matching(test_sentence, bi_grams, tri_grams)

In [72]:
tokens

['tôi', 'chơi', 'đá', 'bóng']

In [73]:
probs, paths = viterbi_initialize(states, tag_counts, A, B, tokens, vocab_dict)
pred = viterbi_backward(probs, paths, new_states)


In [75]:
for word, tag in zip(tokens, pred):
  print(f'{word}/{tag}', end=' ')

tôi/V chơi/V đá/V bóng/V 